In [1]:
import polars as pl
pl.Config.with_columns_kwargs = True

In [124]:
batting = pl.read_csv('Data/lahman baseball data/Batting.csv')
pitching = pl.read_csv('Data/lahman baseball data/Pitching.csv')
fielding = pl.read_csv('Data/lahman baseball data/Fielding.csv')
awards = pl.read_csv('Data/lahman baseball data/AwardsPlayers.csv')
salaries = pl.read_csv('Data/lahman baseball data/Salaries.csv')
awards

playerID,awardID,yearID,lgID,tie,notes
str,str,i64,str,str,str
"""poseybu01""","""Lou Gehrig Memorial Award""",2019,"""ML""",null,null
"""klubeco01""","""Lou Gehrig Memorial Award""",2018,"""ML""",null,null
"""vottojo01""","""Lou Gehrig Memorial Award""",2017,"""ML""",null,null
"""altuvjo01""","""Lou Gehrig Memorial Award""",2016,"""ML""",null,null
"""grandcu01""","""Lou Gehrig Memorial Award""",2015,"""ML""",null,null
…,…,…,…,…,…
"""barnhtu01""","""Gold Glove""",2017,"""NL""",null,"""C"""
"""goldspa01""","""Gold Glove""",2017,"""NL""",null,"""1B"""
"""lemahdj01""","""Gold Glove""",2017,"""NL""",null,"""2B"""


In [12]:
salaries.columns

['yearID', 'teamID', 'lgID', 'playerID', 'salary']

In [109]:
awards.columns

['playerID', 'yearID', 'awardID']

In [146]:
fielding.sort('yearID', descending=True)


playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str
"""abadfe01""",2023,1,"""COL""","""NL""","""P""",6,0,19,0,0,1,0,null,null,null,null,null
"""abbotan01""",2023,1,"""CIN""","""NL""","""P""",21,21,328,1,9,0,1,null,null,null,null,null
"""abbotco01""",2023,1,"""WAS""","""NL""","""P""",22,0,118,0,1,0,1,null,null,null,null,null
"""abramcj01""",2023,1,"""WAS""","""NL""","""SS""",151,148,3884,245,384,22,100,null,null,null,null,null
"""abreual01""",2023,1,"""NYA""","""AL""","""P""",45,0,177,6,4,2,0,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""wrighha01""",1871,1,"""BS1""","""NA""","""P""",9,0,57,1,5,1,0,null,null,null,null,null
"""wrighha01""",1871,1,"""BS1""","""NA""","""SS""",1,1,18,0,3,2,0,null,null,null,null,null
"""yorkto01""",1871,1,"""TRO""","""NA""","""OF""",29,29,750,60,5,11,2,null,null,null,null,null


In [36]:
fielding.columns

['playerID',
 'yearID',
 'stint',
 'teamID',
 'lgID',
 'POS',
 'G',
 'GS',
 'InnOuts',
 'PO',
 'A',
 'E',
 'DP',
 'PB',
 'WP',
 'SB',
 'CS',
 'ZR']

In [ ]:
'case is player, year, pos, league'

In [151]:
awards_filter = (awards
                 .rename({"notes": "pos"})

                 .select(['playerID',
                           'yearID',
                           'awardID',
                           'pos'])
                 .filter(pl.col('awardID') == 'Gold Glove'))

fielding_awards = (fielding
.join(awards_filter, on=['playerID', 'yearID'], how='left')
.group_by(pl.col('yearID'), pl.col('lgID'), pl.col('playerID'), pl.col('awardID'))
.agg(pl.col('InnOuts').sum().alias('InnOuts'),
     pl.col('PO').sum().alias('PO'),
     pl.col('A').sum().alias('A'),
     pl.col('E').sum().alias('E'),
     pl.col('DP').sum().alias('DP'),
     pl.col('PB').sum().alias('PB'),
     pl.col('WP').sum().alias('WP'),
     pl.col('SB').sum().alias('SB'),
     pl.col('CS').sum().alias('CS'),
     pl.col('ZR').sum().alias('ZR')
    )    

 .with_columns(
        pl.when(pl.col('awardID').is_null())
          .then(pl.lit('No'))
          .otherwise(pl.lit('Yes'))
          .alias('Gold Glove?')
    )
 # .with_columns(
 #         pl.when(pl.col('yearID') == '2023')
 #            .then(pl.lit('Validation'))
 #            .otherwise(pl.lit('Training'))
 #                  )
                   
                  )
fielding_awards

yearID,lgID,playerID,awardID,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,Gold Glove?
i64,str,str,str,i64,i64,i64,i64,i64,str,str,str,str,str,str
1990,"""NL""","""quiseda01""",null,20,0,1,0,0,null,null,null,null,null,"""No"""
2012,"""NL""","""sanchbr01""",null,19,0,0,0,0,null,null,null,null,null,"""No"""
2017,"""AL""","""warread01""",null,172,1,10,0,0,null,null,null,null,null,"""No"""
2019,"""AL""","""rondojo02""",null,948,43,80,4,15,null,null,null,null,null,"""No"""
1882,"""AA""","""mountfr01""",null,213,2,31,3,0,null,null,null,null,null,"""No"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1983,"""AL""","""wathajo01""",null,3173,615,58,9,29,null,null,null,null,null,"""No"""
1996,"""NL""","""webstle01""",null,1293,390,25,1,3,null,null,null,null,null,"""No"""
1890,"""NL""","""stockle01""",null,54,10,1,3,0,null,null,null,null,null,"""No"""
